# Des classes pour représenter les données

In [1]:
case class Prenom(sexe: String, prenom: String, annee: Int, codeDept: Int, nombre: Int)
case class Dept(region: Int, codeDept: String, chefLieu: String, typeNom: Int, nom: String, nomEnrichi: String)

# Chargement des données
1. Créer un dataset pour les prénoms

In [4]:
// Pour les conversions implicites de RDDs vers DataFrames
val sparkRO = spark // bricolage pour que cela fonctionne dans le notebool (inutile sinon)
import sparkRO.implicits._

In [21]:
val prenoms = spark.read.load("prenomsParDeptsEtAnnees.parquet").as[Prenom]
prenoms.createOrReplaceTempView("prenoms")
prenoms.show

+----+-------------+------+--------+-----+
|sexe|       prenom|nombre|codeDept|annee|
+----+-------------+------+--------+-----+
|   1|       WASSIL|     5|      69| 2009|
|   1|     ZACHARIA|     3|      69| 2009|
|   2|   LEOPOLDINE|     4|      69| 2009|
|   2|        LINDA|     9|      69| 2009|
|   2|MARIE-TH�R�SE|    41|      57| 1961|
|   2|   ROSE-MARIE|    14|      57| 1961|
|   2|    ABIGAELLE|     5|      97| 2012|
|   2|        A�CHA|     5|      97| 2012|
|   2|      JOHANNA|    18|      97| 2012|
|   1|         STAN|     3|      97| 2008|
|   1|       YANAEL|     5|      97| 2008|
|   2|        GRACE|     5|      97| 2008|
|   1|       SECKOU|     3|      93| 2011|
|   1|        SOREN|     6|      93| 2011|
|   2|        DJENA|     3|      93| 2011|
|   1|     ANASTASE|    10|      97| 1955|
|   1|        DONAT|     4|      97| 1955|
|   1|    FLORIBERT|     3|      97| 1955|
|   1|       GILDAS|     4|      97| 1955|
|   1|      JOAQUIN|     3|      75| 1955|
+----+-----

1. Faire de même pour les départements

In [22]:
val depts = spark.read.load("depts.parquet").as[Dept]
depts.createOrReplaceTempView("depts")
depts.show

+------+--------+--------+-------+--------------------+--------------------+
|region|codeDept|chefLieu|typeNom|                 nom|          nomEnrichi|
+------+--------+--------+-------+--------------------+--------------------+
|    44|      52|   52121|      3|         HAUTE-MARNE|         Haute-Marne|
|    52|      53|   53130|      3|             MAYENNE|             Mayenne|
|    44|      54|   54395|      0|  MEURTHE-ET-MOSELLE|  Meurthe-et-Moselle|
|    44|      55|   55029|      3|               MEUSE|               Meuse|
|    53|      56|   56260|      2|            MORBIHAN|            Morbihan|
|    44|      57|   57463|      3|             MOSELLE|             Moselle|
|    27|      58|   58194|      3|              NIEVRE|              Ni�vre|
|    32|      59|   59350|      2|                NORD|                Nord|
|    32|      60|   60057|      5|                OISE|                Oise|
|    28|      61|   61001|      5|                ORNE|                Orne|

# Interroger les données

* La documentation sur Spark SQL est disponible [ici](https://spark.apache.org/docs/latest/sql-programming-guide.html) (de même pour [Dataset](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset))
* Pour chaque question, vous donnerez la réponse avec le DSL et en SQL

1. Donner, pour chaque prénom, son nombre d'occurences

## DSL

In [23]:
prenoms.select($"prenom", $"nombre").groupBy($"prenom").sum("nombre").show

+------------+-----------+
|      prenom|sum(nombre)|
+------------+-----------+
|       NORIA|          3|
|        ANNA|         90|
|        JADE|          7|
|     L�ANDRE|          3|
|      ELIANE|         75|
|     MATTHEW|          4|
|      SALOM�|         46|
|    ALPHONSE|         72|
|      FRANCK|        423|
|       MORAD|          8|
|       MARAM|          3|
|       GRACE|          5|
|SAINTE-CROIX|          6|
|    FERNANDA|          3|
|        JO�L|        110|
|      ANNICK|         75|
|      LUCILE|          3|
|      SANDIE|          5|
|      AMEDEE|         10|
|       FABIO|         29|
+------------+-----------+
only showing top 20 rows



## SQL

In [24]:
spark.sql("SELECT prenom, SUM(nombre) FROM prenoms GROUP BY prenom").show()

+------------+-----------+
|      prenom|sum(nombre)|
+------------+-----------+
|       NORIA|          3|
|        ANNA|         90|
|        JADE|          7|
|     L�ANDRE|          3|
|      ELIANE|         75|
|     MATTHEW|          4|
|      SALOM�|         46|
|    ALPHONSE|         72|
|      FRANCK|        423|
|       MORAD|          8|
|       MARAM|          3|
|       GRACE|          5|
|SAINTE-CROIX|          6|
|    FERNANDA|          3|
|        JO�L|        110|
|      ANNICK|         75|
|      LUCILE|          3|
|      SANDIE|          5|
|      AMEDEE|         10|
|       FABIO|         29|
+------------+-----------+
only showing top 20 rows



1. Donner le nombre total de naissances avec un prénom féminin

## DSL

In [37]:
prenoms.filter($"sexe" === "2").groupBy().sum("nombre").show

+-----------+
|sum(nombre)|
+-----------+
|      33273|
+-----------+



## SQL

In [40]:
spark.sql("SELECT SUM(nombre) FROM prenoms WHERE sexe = \"2\"").show()

+-----------+
|sum(nombre)|
+-----------+
|      33273|
+-----------+



1. Donner l'effectif maximal et minimal par prénom

## DSL

In [43]:
prenoms.select($"prenom", $"nombre" as "nbmin", $"nombre" as "nbmax").groupBy($"prenom").agg(Map("nbmin" -> "min", "nbmax" -> "max")).show

+------------+----------+----------+
|      prenom|min(nbmin)|max(nbmax)|
+------------+----------+----------+
|       NORIA|         3|         3|
|        ANNA|         3|        55|
|        JADE|         7|         7|
|     L�ANDRE|         3|         3|
|      ELIANE|         6|        39|
|     MATTHEW|         4|         4|
|      SALOM�|         6|        28|
|    ALPHONSE|        24|        48|
|      FRANCK|         3|       203|
|       MORAD|         8|         8|
|       MARAM|         3|         3|
|       GRACE|         5|         5|
|SAINTE-CROIX|         6|         6|
|    FERNANDA|         3|         3|
|        JO�L|         3|        48|
|      ANNICK|         9|        66|
|      LUCILE|         3|         3|
|      SANDIE|         5|         5|
|      AMEDEE|         5|         5|
|       FABIO|         3|        26|
+------------+----------+----------+
only showing top 20 rows



## SQL

In [44]:
spark.sql("SELECT prenom, MIN(nombre), MAX(nombre) FROM prenoms GROUP BY prenom").show()

+------------+-----------+-----------+
|      prenom|min(nombre)|max(nombre)|
+------------+-----------+-----------+
|       NORIA|          3|          3|
|        ANNA|          3|         55|
|        JADE|          7|          7|
|     L�ANDRE|          3|          3|
|      ELIANE|          6|         39|
|     MATTHEW|          4|          4|
|      SALOM�|          6|         28|
|    ALPHONSE|         24|         48|
|      FRANCK|          3|        203|
|       MORAD|          8|          8|
|       MARAM|          3|          3|
|       GRACE|          5|          5|
|SAINTE-CROIX|          6|          6|
|    FERNANDA|          3|          3|
|        JO�L|          3|         48|
|      ANNICK|          9|         66|
|      LUCILE|          3|          3|
|      SANDIE|          5|          5|
|      AMEDEE|          5|          5|
|       FABIO|          3|         26|
+------------+-----------+-----------+
only showing top 20 rows



1. Donner, pour chaque nom de département, le nombre moyen de prénoms depuis l'année 2000

## DSL

In [66]:
prenoms.filter($"annee" >= 2000).join(depts, depts.col("codeDept") === prenoms.col("codeDept")).select($"nom", $"nombre").groupBy($"nom").agg("nombre" -> "avg").sort($"nom").show

+--------------------+------------------+
|                 nom|       avg(nombre)|
+--------------------+------------------+
|                 AIN|               6.5|
|               AISNE| 7.714285714285714|
|              ALLIER|               7.6|
|ALPES-DE-HAUTE-PR...| 4.333333333333333|
|     ALPES-MARITIMES|              14.0|
|             ARDECHE| 9.666666666666666|
|            ARDENNES|               3.0|
|              ARIEGE|               5.0|
|                AUBE|              10.0|
|                AUDE|              11.4|
|             AVEYRON|              8.75|
|            BAS-RHIN|11.181818181818182|
|    BOUCHES-DU-RHONE|              15.0|
|            CALVADOS|               4.0|
|              CANTAL|              7.25|
|            CHARENTE| 6.857142857142857|
|   CHARENTE-MARITIME|              14.0|
|                CHER| 5.833333333333333|
|             CORREZE| 9.833333333333334|
|           COTE-D'OR|37.285714285714285|
+--------------------+------------

## SQL

In [65]:
spark.sql("SELECT nom, AVG(nombre) FROM prenoms p JOIN depts d ON p.codeDept = d.codeDept WHERE annee >= 2000 GROUP BY nom ORDER BY nom").show()

+--------------------+------------------+
|                 nom|       avg(nombre)|
+--------------------+------------------+
|                 AIN|               6.5|
|               AISNE| 7.714285714285714|
|              ALLIER|               7.6|
|ALPES-DE-HAUTE-PR...| 4.333333333333333|
|     ALPES-MARITIMES|              14.0|
|             ARDECHE| 9.666666666666666|
|            ARDENNES|               3.0|
|              ARIEGE|               5.0|
|                AUBE|              10.0|
|                AUDE|              11.4|
|             AVEYRON|              8.75|
|            BAS-RHIN|11.181818181818182|
|    BOUCHES-DU-RHONE|              15.0|
|            CALVADOS|               4.0|
|              CANTAL|              7.25|
|            CHARENTE| 6.857142857142857|
|   CHARENTE-MARITIME|              14.0|
|                CHER| 5.833333333333333|
|             CORREZE| 9.833333333333334|
|           COTE-D'OR|37.285714285714285|
+--------------------+------------